Création de 3 fichiers de production
prodA pour le pays A, prodB pour le pays B, prodC pour le pays C

Les fichiers contiennent en réalité les quantité d'électricité produites (éolien + solaires) du programme ENR_PL_classique découpées en 3 trimestres.

In [11]:
import numpy as np
import pandas as pd

### Choix

In [12]:
N = 5 # Nombre de fichiers de production à créer (ie de de pays dans la simulation)

In [13]:
# Chargement des données
df_solar = pd.read_csv('solar.csv')
df_wind = pd.read_csv('wind_onshore.csv')
df_demand = pd.read_csv('demand2050_ADEME.csv', header=None)
df_demand.columns = ["heures", "demande"]

# Calcul consommation annuelle
annual_demand = df_demand['demande'].sum()

# Calcul production potentielle pour 1GW installé
annual_solar_per_gw = df_solar['facteur_charge'].sum()
annual_wind_per_gw = df_wind['facteur_charge'].sum()

# Pour avoir 100% de surproduction et un mix 50-50
target_production = annual_demand * 3.5
wind_capacity = target_production/(2 * annual_wind_per_gw)
solar_capacity = target_production/(2 * annual_solar_per_gw)


Création de 3 bdd de production différentes, fictives, pour les pays A, B et C

In [14]:
nbHeures = len(df_demand)  # nombre d'heures
T = nbHeures//N

debutPer = 0

for i in range(N) :
    nom = chr(65 + i)  # Nom de la période (A, B, C, D, ...), 65 = 'A' en ASCII

    solar_profile = df_solar['facteur_charge'].values[debutPer:debutPer+T]
    wind_profile = df_wind['facteur_charge'].values[debutPer:debutPer+T]
    demand = df_demand['demande'].values[debutPer:debutPer+T]

    # Production ENR horaire
    prod = {t: wind_capacity * wind_profile[t] + solar_capacity * solar_profile[t] for t in range(len(solar_profile))}

    # Création de la base de données
    df = pd.DataFrame(list(prod.items()), columns=['Heure', 'Production_Elec'])
    df.to_csv(f'prod{nom}.csv', index=False, encoding='utf-8') # c'est ici seulement qu'apparait le nom du fichier, distinct pour chaque pays

    debutPer += T
